In [1]:
import tensorflow as tf

/home/phunc20/.config/miniconda3/envs/homl-1e/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/phunc20/.config/miniconda3/envs/homl-1e/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/phunc20/.config/miniconda3/envs/homl-1e/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/phunc20

In [3]:
import numpy as np
tf.__version__, np.__version__

('1.13.1', '1.19.2')

In [4]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

Instructions for updating:
Colocations handled automatically by placer.


It's funny that `ageron` has again chosen the result of the computation to be `42`:

In [43]:
3*3*4 + 4 + 2

42

**No computation being performed** yet. Only creates computation graph.

To _evaluate_ this graph, we need to
- Open a TensorFlow _session_
- Use the session to initialize the variables and evaluate `f`

A TensorFlow session takes care of placing the operations onto _devices_ such as CPUs and GPUs.


In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
# sess.close() frees up resources
sess.close()

42


**(?1)** What'd happen if we run `sess.run()` after having closed the session?

**(R1)** We simply aren't allowed to do that.

In [9]:
g = x - y
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(g)
sess.close()
g, result

(<tf.Tensor 'sub_3:0' shape=() dtype=int32>, -1)

### `with` block does the same thing

In [10]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

In [12]:
with tf.Session() as sess:
    sess.run(x.initializer)
    sess.run(y.initializer)
    result = sess.run(f)
result

42

In [14]:
with tf.Session() as sess:
    tf.get_default_session().run(x.initializer)
    tf.get_default_session().run(y.initializer)
    result = tf.get_default_session().run(f)
result

42

- `tf.get_default_session()` has to be used inside a `with tf.Session() as sess:` block
- otherwise it will be just a `None`.

In [20]:
print(tf.get_default_session())

None


In [16]:
tf.get_default_session() is None

True

In [19]:
with tf.Session() as sess:
    print(tf.get_default_session())

In [22]:
with tf.Session() as sess:
    print("tf.get_default_session() is sess:", tf.get_default_session() is sess)

tf.get_default_session() is sess: True


### One more convenience function: `global_variables_initializer()`
With this, we don't have to manually type `.initializer` each time. Note that it does not actually perform the initialization immediately; instead, it creates a node in the graph that will initialize all variables when it is run.

In [37]:
init = tf.global_variables_initializer()  # prepare an init node
with tf.Session() as sess:
    init.run()  # actually initialize all the variables
    result = f.eval()
result

42

### `tf.InteractiveSession()`
is often used in `ipython` and Jupyter notebooks. It is similar to the `with` block above, but one has to remember to close it at the end.

In [38]:
"init" in locals()

True

In [39]:
if "init" in locals():
    del init

In [40]:
"init" in locals()

False

In [42]:
init = tf.global_variables_initializer()  # prepare an init node
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42
